In [1]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)


In [ ]:
from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o-2024-11-20")

llm = ChatOpenAI(model="gpt-4o")

In [15]:
from langchain.agents import initialize_agent, load_tools, AgentType

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [16]:
agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handle_parsing_errors=True, verbose=True) # 제로샷 ReAct 에이전트

agent.invoke("소나무 옮겨심기 좋은 계절은?") # 에이전트에게 질문을 전달하고 답변을 생성하도록 실행



> Entering new AgentExecutor chain...
I need to find out the best season for transplanting pine trees.
Action: wikipedia
Action Input: Search for "best season for transplanting pine trees"
Observation: No good Wikipedia Search Result was found
Thought:I should try a different search query that is more specific.
Action: wikipedia
Action Input: Search for "transplanting pine trees season"
Observation: No good Wikipedia Search Result was found
Thought:I should try searching for "best season for pine tree transplant" instead.
Action: wikipedia
Action Input: Search for "best season for pine tree transplant"
Observation: No good Wikipedia Search Result was found
Thought:I should try searching for "best season for transplanted pine trees" instead.
Action: wikipedia
Action Input: Search for "best season for transplanted pine trees"
Observation: No good Wikipedia Search Result was found
Thought:I should try a different approach and search for general information on when to transplant pine tre

ValueError: unknown format from LLM: I'm sorry, but the question provided does not seem to be a math problem that can be translated into a mathematical expression for execution using Python's numexpr library.

In [17]:
# Conversational ReAct (Memory 포함) 에이전트

from langchain.memory import ConversationBufferMemory

tools = load_tools(["llm-math"],  llm=llm) # 에이전트가 사용할 도구 지정, 수학 관련 문제를 해결하는 도구

memory = ConversationBufferMemory(memory_key="chat_history") # 대화 기록을 저장할 키를 지정

conversational_agent = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True, # 디버깅 목적으로 에이전트의 내부 동작을 출력
    max_iterations=3, # 에이전트가 질문을 해결하기 위해 도구를 사용할 수 있는 최대 반복 횟수를 3으로 지
    memory=memory,)

conversational_agent.invoke("아이유는 누구이며 2025년 현재 나이는 몇살이야?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: 아이유는 대한민국의 가수, 배우이며 실명은 이지은입니다. 2025년 현재 아이유는 현재 33살이 되실 것입니다.

> Finished chain.


{'input': '아이유는 누구이며 2025년 현재 나이는 몇살이야?',
 'chat_history': '',
 'output': '아이유는 대한민국의 가수, 배우이며 실명은 이지은입니다. 2025년 현재 아이유는 현재 33살이 되실 것입니다.'}

In [18]:
#Self Ask With Search Agent 사용
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer

In [19]:
tools = [TavilyAnswer(max_results=5, name="Intermediate Answer")] # 도구 이름을 설정하여 나중에 에이전트가 이 도구를 참조할 때 사용

prompt = hub.pull("hwchase17/self-ask-with-search") # 랭체인에서 제공하는 프롬프트 허브에서 self-ask-with-search 프롬프트를 가져옵니

agent = create_self_ask_with_search_agent(llm, tools, prompt) # "Self Ask With Search" 기능을 구현하는 에이전트를 생성

agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True) #AgentExecutor 이용하여 에이전트 사용

agent_executor.invoke({"input": "AI Agent를 세 문장으로 설명해줘"})

{'input': 'AI Agent를 세 문장으로 설명해줘',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [11]:
# ReAct document store
from langchain.docstore.wikipedia import Wikipedia # 문서 저장소로 위키피디아 이용
from langchain.agents import Tool
from langchain.agents.react.base import DocstoreExplorer # 문서가 저장된 데이터베이스(위키피디아)를 탐색하는 도구

docstore=DocstoreExplorer(Wikipedia())

tools = [
  Tool(
    name="Search",
    func=docstore.search,  # 용어에 대한 키워드 기반의 검색, 사용자가 정확히 무엇을 찾고자 하는지 모를 때 유용
    description="docstore에서 용어를 검색하세요.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup, # 고유하게 식별할 수 있는 키를 이용한 검색, 특정 문서(또는 항목)를 정확하게 찾을 때 사용
    description="docstore에서 용어 검색.",
  )
] # 에이전트가 사용할 수 있는 도구를 정의

/tmp/ipykernel_24999/3100421358.py:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  docstore=DocstoreExplorer(Wikipedia())


In [12]:
react = initialize_agent(tools = tools,
                         llm = llm,
                         agent = AgentType.REACT_DOCSTORE, # 에이전트 유형을 ReAct document store로 지정
                         handle_parsing_errors=True, # 에이전트가 작업을 수행하는 동안 발생하는 구문 오류(Parsing Errors)를 처리
                         max_iterations=1, # 에이전트가 최대 몇 번까지 작업을 반복할지 지정
                         max_execution_time=1, # 에이전트가 작업을 수행하는 최대 시간을 초 단위로 설정
                         verbose = True,) # 랭체인 에이전트 초기화

In [13]:
def query_data(query):
    try:
        response = react.invoke(query)  # 에이전트에게 쿼리 전달
        print("----------------------", response)  # 결과 출력(디버깅용)
        return response  # 검색 결과 반환
    except Exception as e:
        print(f"Error: {e}")  # 에러 메시지를 출력
        raise  # 에러 다시 발생(추가 디버깅 용도)

query = "밍크 선인장 키우는 방법은?"
response = query_data(query)
print(response['output']) # 에이전트의 최종 답변 출력



> Entering new AgentExecutor chain...
Could not parse LLM Output: 밍크 선인장을 키우는 방법에 대해 알려드리겠습니다:

1. **적절한 빛**: 밍크 선인장은 밝은 빛을 좋아하지만, 직사광선을 피하는 것이 좋습니다. 실내에서는 햇빛이 잘 드는 창가 쪽에 두는 것이 이상적입니다.

2. **물 주기**: 선인장은 과도한 물을 싫어합니다. 흙이 완전히 건조된 후에 물을 주는 것이 중요합니다. 일반적으로 봄과 여름에는 2~3주에 한 번, 가을과 겨울에는 물 주는 빈도를 더 줄이세요.

3. **흙 배합**: 배수가 잘 되는 선인장 전용 흙을 사용하는 것이 좋습니다. 배수가 잘 되지 않으면 뿌리가 썩을 위험이 있습니다.

4. **온도와 습도**: 밍크 선인장은 따뜻하고 건조한 환경을 좋아합니다. 일반적인 실내 온도(약 18~26°C)에서 잘 자랍니다. 겨울에는 너무 추운 환경을 피하세요.

5. **비료**: 선인장 전용 비료를 사용하여 봄과 여름에 한 달에 한 번 정도 비료를 주면 성장에 도움이 됩니다. 가을과 겨울에는 비료를 주지 않아도 됩니다.

6. **분갈이**: 밍크 선인장은 천천히 자라므로 2~3년에 한 번씩 분갈이를 해주세요. 분갈이 시 뿌리를 조심스럽게 다루고 새로운 흙을 사용하세요.

7. **해충 관리**: 선인장은 해충에 강한 편이지만, 간혹 깍지벌레 등의 해충이 생길 수 있습니다. 살충제를 사용하거나 면봉에 알코올을 묻혀 닦아내세요.

위의 관리 방법을 따르면 밍크 선인장을 건강하게 키우실 수 있을 것입니다!
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:

> Finished chain.
---------------------- {'input': '